# Import library

In [34]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [35]:
import torch
from torchvision import transforms
from torchvision import models
from PIL import Image
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Read image and label

In [3]:
# Load image
path = '/content/drive/MyDrive/EE3063/HW7/image/'
file_name = os.listdir(path)
images = [Image.open(path + name) for name in file_name]
print(len(images))

100


In [4]:
#Get labels
labels = [name.split(name[-6])[0] for name in file_name]
print(labels)

['yawl', 'yawl', 'yawl', 'yawl', 'table lamp', 'table lamp', 'table lamp', 'table lamp', 'spotlight', 'spotlight', 'spotlight', 'spotlight', 'monastery', 'monastery', 'monastery', 'monastery', 'black grouse', 'cuirass', 'black grouse', 'cuirass', 'cuirass', 'cuirass', 'black grouse', 'black grouse', 'Angora', 'Angora', 'Angora', 'Angora', 'medicine chest', 'medicine chest', 'medicine chest', 'medicine chest', 'gasmask', 'gasmask', 'gasmask', 'gasmask', 'lacewing', 'lacewing', 'lacewing', 'lacewing', 'anemone fish', 'anemone fish', 'anemone fish', 'anemone fish', 'great grey owl', 'great grey owl', 'great grey owl', 'great grey owl', 'gondola', 'trolleybus', 'gondola', 'trolleybus', 'trolleybus', 'gondola', 'trolleybus', 'gondola', 'jellyfish', 'jellyfish', 'jellyfish', 'jellyfish', 'viaduct', 'viaduct', 'viaduct', 'viaduct', 'vulture', 'vulture', 'vulture', 'vulture', 'chocolate sauce', 'chocolate sauce', 'chocolate sauce', 'chocolate sauce', 'pier', 'pier', 'pier', 'pier', 'recreation

# Prepare data

In [5]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [6]:
transform_imgs = torch.empty(0, 3, 224, 224)
for img in images:
  tf_img = preprocess(img)
  tensor_img = torch.unsqueeze(tf_img, 0)
  transform_imgs = torch.cat((transform_imgs, tensor_img), dim = 0)

print(transform_imgs.shape)

torch.Size([100, 3, 224, 224])


# Load model

## Alexnet

In [11]:
alexnet = models.alexnet(pretrained=True)
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

## Resnet18

In [12]:
resnet18 = models.resnet18(pretrained=True)
print(resnet18)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Inception v3

In [13]:
inception = models.inception_v3(pretrained=True)
print(inception)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

# Evaluation

## Alexnet

In [14]:
model = alexnet
model.eval()

predictions = model(transform_imgs)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [15]:
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
output = torch.nn.functional.softmax(predictions, dim=-1)

In [16]:
# Read the categories
with open("/content/drive/MyDrive/EE3063/HW7/imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

In [17]:
# Show top categories per image
top5_prob, top5_id = torch.topk(output, 5)
true = 0
error = []

for i in range(top5_id.shape[0]):
  if categories[top5_id[i][0]].startswith(labels[i]):
    true += 1
  elif categories[top5_id[i][1]].startswith(labels[i]):
    true += 1
  elif categories[top5_id[i][2]].startswith(labels[i]):
    true += 1
  elif categories[top5_id[i][3]].startswith(labels[i]):
    true += 1
  elif categories[top5_id[i][4]].startswith(labels[i]):
    true += 1
  else:
    error.append(i)


print('Top 5 error rate: %.2f' %(100*(1-true/top5_id.shape[0])))

Top 5 error rate: 12.00


In [18]:
# Show top categories per image
top1_prob, top1_id = torch.topk(output, 1)
true = 0

for i in range(top1_id.shape[0]):
  if categories[top1_id[i][0]].startswith(labels[i]):
    true += 1

print('Top 1 error rate: %.2f' %(100*(1-true/top1_id.shape[0])))

Top 1 error rate: 41.00


## Resnet18

In [19]:
model = resnet18
model.eval()

predictions = model(transform_imgs)

In [20]:
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
output = torch.nn.functional.softmax(predictions, dim=-1)

In [21]:
# Show top categories per image
top5_prob, top5_id = torch.topk(output, 5)
true = 0
error = []

for i in range(top5_id.shape[0]):
  if categories[top5_id[i][0]].startswith(labels[i]):
    true += 1
  elif categories[top5_id[i][1]].startswith(labels[i]):
    true += 1
  elif categories[top5_id[i][2]].startswith(labels[i]):
    true += 1
  elif categories[top5_id[i][3]].startswith(labels[i]):
    true += 1
  elif categories[top5_id[i][4]].startswith(labels[i]):
    true += 1
  else:
    error.append(i)


print('Top 5 error rate: %.2f' %(100*(1-true/top5_id.shape[0])))

Top 5 error rate: 4.00


In [22]:
# Show top categories per image
top1_prob, top1_id = torch.topk(output, 1)
true = 0

for i in range(top1_id.shape[0]):
  if categories[top1_id[i][0]].startswith(labels[i]):
    true += 1

print('Top 1 error rate: %.2f' %(100*(1-true/top1_id.shape[0])))

Top 1 error rate: 26.00


## Inception

In [23]:
model = inception
model.eval()

predictions = model(transform_imgs)

In [27]:
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
output = torch.nn.functional.softmax(predictions, dim=-1)

In [28]:
# Show top categories per image
top5_prob, top5_id = torch.topk(output, 5)
true = 0
error = []

for i in range(top5_id.shape[0]):
  if categories[top5_id[i][0]].startswith(labels[i]):
    true += 1
  elif categories[top5_id[i][1]].startswith(labels[i]):
    true += 1
  elif categories[top5_id[i][2]].startswith(labels[i]):
    true += 1
  elif categories[top5_id[i][3]].startswith(labels[i]):
    true += 1
  elif categories[top5_id[i][4]].startswith(labels[i]):
    true += 1
  else:
    error.append(i)

print('Top 5 error rate: %.2f' %(100*(1-true/top5_id.shape[0])))

Top 5 error rate: 11.00


In [29]:
# Show top categories per image
top1_prob, top1_id = torch.topk(output, 1)
true = 0

for i in range(top1_id.shape[0]):
  if categories[top1_id[i][0]].startswith(labels[i]):
    true += 1

print('Top 1 error rate: %.2f' %(100*(1-true/top1_id.shape[0])))

Top 1 error rate: 33.00


# Plot result

In [ ]:
# View top 5 class for each image
for i in range(100):
  print('Image %s' %(file_name[i]))
  for j in range(5):
    print(' %s   with accuracy is %.4f' %(categories[top5_id[i][j]], top5_prob[i][j]))

In [ ]:
# Choose image
i = 6

# Plot
plt.rcdefaults()
fig, ax = plt.subplots()
label = []
# Example data
for j in range(5):
  label.append(categories[top5_id[i][j]])

y_pos = np.arange(len(label))
prob = top5_prob[i].detach().numpy()

ax.barh(y_pos, 100*prob, align='center', color=['red', 'red', 'red', 'red', 'green'])
ax.set_yticks(y_pos)
ax.set_yticklabels(label)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_title(labels[i])

plt.show()